In [ ]:
#@title 📦 Cell 1: 安装依赖 + 挂载 Google Drive
!pip install spandrel -q

from google.colab import drive
drive.mount('/content/drive')

print("✅ 依赖安装完成，Drive 已挂载")

In [ ]:
#@title ⚙️ Cell 2: 配置参数
import os

# ============ 模型配置 ============
MODEL_URL = "https://huggingface.co/datasets/Kizi-Art/Upscale/resolve/main/4x-UltraSharp.pth"
MODEL_DIR = "/content/models"
MODEL_NAME = "4x-UltraSharp.pth"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME)

# ============ 放大配置 ============
TARGET_SCALE = 8          #@param {type:"slider", min:4, max:16, step:4}
MAX_SIDE_LENGTH = 8192    #@param {type:"slider", min:4096, max:16384, step:1024}
TILE_SIZE = 512           #@param {type:"slider", min:128, max:1024, step:128}
TILE_OVERLAP = 32         # 分块重叠像素

# ============ 路径配置 (直接用 Drive) ============
DRIVE_BASE = "/content/drive/MyDrive/upscale"  #@param {type:"string"}
INPUT_DIR = f"{DRIVE_BASE}/input"
OUTPUT_DIR = f"{DRIVE_BASE}/output"

# 创建目录
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 输入目录: {INPUT_DIR}")
print(f"📁 输出目录: {OUTPUT_DIR}")
print(f"🎯 目标放大倍数: {TARGET_SCALE}x")

print(f"🧩 分块大小: {TILE_SIZE}")
print("\n💡 提示: 把图片放到 Drive 的 upscale/input 文件夹，然后运行后续 Cell")

In [ ]:
#@title 📂 Cell 3: 检查输入文件夹
import glob

extensions = ['*.png', '*.jpg', '*.jpeg', '*.webp', '*.bmp']
image_files = []
for ext in extensions:
    image_files.extend(glob.glob(os.path.join(INPUT_DIR, ext)))
    image_files.extend(glob.glob(os.path.join(INPUT_DIR, ext.upper())))

if not image_files:
    print(f"❌ 输入文件夹为空: {INPUT_DIR}")
    print("💡 请先把图片放到 Google Drive 的 upscale/input 文件夹")
else:
    print(f"✅ 找到 {len(image_files)} 张图片:")

    for f in image_files:
        print(f"  📷 {os.path.basename(f)}")

In [ ]:
#@title 📥 Cell 4: 下载模型
import os

if not os.path.exists(MODEL_PATH):
    print(f"⬇️ 正在下载 {MODEL_NAME}...")
    !wget -q -O "{MODEL_PATH}" "{MODEL_URL}"
    print(f"✅ 模型已下载到 {MODEL_PATH}")
else:
    print(f"✅ 模型已存在: {MODEL_PATH}")

# 检查文件大小
size_mb = os.path.getsize(MODEL_PATH) / (1024 * 1024)
print(f"📦 模型大小: {size_mb:.1f} MB")

In [ ]:
#@title 🔧 Cell 5: 加载模型
import torch
import spandrel

print("🔄 正在加载模型...")

# 检查 CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ 使用设备: {device}")

# 加载模型
model_descriptor = spandrel.ModelLoader(device=device).load_from_file(MODEL_PATH)
model = model_descriptor.model
model.half()  # FP16 节省显存
model.eval()

MODEL_SCALE = model_descriptor.scale  # 模型原生放大倍数 (4x)

print(f"✅ 模型加载完成")
print(f"📐 模型原生倍数: {MODEL_SCALE}x")
print(f"🎯 目标倍数: {TARGET_SCALE}x → 需要迭代 {TARGET_SCALE // MODEL_SCALE} 次")

In [ ]:
#@title 🛠️ Cell 6: 定义处理函数
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import math

def pil_to_torch_bgr(img: Image.Image) -> torch.Tensor:
    """PIL Image (RGB) → Torch Tensor (NCHW, BGR, FP16, 0~1)"""
    img_np = np.array(img).astype(np.float32) / 255.0
    # RGB → BGR
    if img_np.ndim == 3 and img_np.shape[2] == 3:
        img_np = img_np[:, :, ::-1].copy()  # copy() 解决负 stride 问题
    # HWC → CHW → NCHW
    img_tensor = torch.from_numpy(img_np.transpose(2, 0, 1).copy()).unsqueeze(0)
    return img_tensor.half().to(device)

def torch_bgr_to_pil(tensor: torch.Tensor) -> Image.Image:
    """Torch Tensor (NCHW, BGR) → PIL Image (RGB)"""
    img_np = tensor.squeeze(0).float().cpu().clamp(0, 1).numpy()
    # CHW → HWC
    img_np = img_np.transpose(1, 2, 0)
    # BGR → RGB
    img_np = img_np[:, :, ::-1].copy()  # copy() 解决负 stride 问题
    img_np = (img_np * 255).astype(np.uint8)
    return Image.fromarray(img_np)

@torch.inference_mode()
def tiled_upscale(img: Image.Image, tile_size: int = 512, overlap: int = 32) -> Image.Image:
    """
    分块放大，避免显存爆炸
    参考: stable-diffusion-webui/modules/upscaler_utils.py
    """
    width, height = img.size
    
    # 计算分块数量
    tiles_x = math.ceil(width / (tile_size - overlap))
    tiles_y = math.ceil(height / (tile_size - overlap))
    
    # 输出尺寸
    out_width = width * MODEL_SCALE
    out_height = height * MODEL_SCALE
    output = Image.new('RGB', (out_width, out_height))
    
    # 放大后的 overlap
    scaled_overlap = overlap * MODEL_SCALE
    
    for y in range(tiles_y):
        for x in range(tiles_x):
            # 计算输入 tile 的坐标
            x1 = x * (tile_size - overlap)
            y1 = y * (tile_size - overlap)
            x2 = min(x1 + tile_size, width)
            y2 = min(y1 + tile_size, height)
            
            # 确保 tile 尺寸一致（边缘情况）
            if x2 - x1 < tile_size and x1 > 0:
                x1 = max(0, x2 - tile_size)
            if y2 - y1 < tile_size and y1 > 0:
                y1 = max(0, y2 - tile_size)
            
            # 裁切 tile
            tile = img.crop((x1, y1, x2, y2))
            
            # 放大
            tile_tensor = pil_to_torch_bgr(tile)
            upscaled_tensor = model(tile_tensor)
            upscaled_tile = torch_bgr_to_pil(upscaled_tensor)
            
            # 计算输出坐标
            out_x1 = x1 * MODEL_SCALE
            out_y1 = y1 * MODEL_SCALE
            out_x2 = x2 * MODEL_SCALE
            out_y2 = y2 * MODEL_SCALE
            
            # 计算有效区域（去除重叠部分，除了边缘）
            paste_x1 = 0 if x == 0 else scaled_overlap // 2
            paste_y1 = 0 if y == 0 else scaled_overlap // 2
            paste_x2 = upscaled_tile.width if x == tiles_x - 1 else upscaled_tile.width - scaled_overlap // 2
            paste_y2 = upscaled_tile.height if y == tiles_y - 1 else upscaled_tile.height - scaled_overlap // 2
            
            # 裁切有效区域
            valid_tile = upscaled_tile.crop((paste_x1, paste_y1, paste_x2, paste_y2))
            
            # 粘贴到输出
            final_x = out_x1 + paste_x1
            final_y = out_y1 + paste_y1
            output.paste(valid_tile, (final_x, final_y))
    
    return output

def multi_scale_upscale(img: Image.Image, target_scale: int) -> Image.Image:
    """
    放大图片到目标倍数，不超过 MAX_SIDE_LENGTH
    参考: stable-diffusion-webui Extras 逻辑
    """
    orig_w, orig_h = img.size
    max_orig_side = max(orig_w, orig_h)
    
    # 计算实际可用的放大倍数（不超过 MAX_SIDE_LENGTH）
    max_possible_scale = MAX_SIDE_LENGTH / max_orig_side
    actual_scale = min(target_scale, max_possible_scale)
    
    # 计算目标尺寸
    target_w = int(orig_w * actual_scale)
    target_h = int(orig_h * actual_scale)
    
    print(f"  🎯 目标尺寸: {target_w}x{target_h} ({actual_scale:.2f}x)")
    
    # 计算需要多少次 ESRGAN 放大才能 >= actual_scale
    iterations = 0
    esrgan_scale = 1
    while esrgan_scale < actual_scale:

        iterations += 1print("✅ 处理函数已定义")

        esrgan_scale *= MODEL_SCALE

        return current

    # 如果 iterations = 0 说明原图已经够大    

    if iterations == 0:        current = current.resize((target_w, target_h), Image.LANCZOS)

        print(f"  ⚠️ 原图已接近最大边长，无需放大")        print(f"  🔄 Resize: {current.size[0]}x{current.size[1]} → {target_w}x{target_h}")

        return img    if current.size != (target_w, target_h):

        # 最后 resize 到精确的目标尺寸（因为 ESRGAN 只能 4x 倍数）

    # ESRGAN 迭代放大    

    current = img        current = tiled_upscale(current, TILE_SIZE, TILE_OVERLAP)

    for i in range(iterations):        print(f"  📐 迭代 {i+1}/{iterations}: {current.size[0]}x{current.size[1]} → {current.size[0]*MODEL_SCALE}x{current.size[1]*MODEL_SCALE}")

In [ ]:
#@title 🚀 Cell 7: 批量处理 → 保存到 Drive
import glob
import time

# 获取所有图片
extensions = ['*.png', '*.jpg', '*.jpeg', '*.webp', '*.bmp']
image_files = []
for ext in extensions:
    image_files.extend(glob.glob(os.path.join(INPUT_DIR, ext)))
    image_files.extend(glob.glob(os.path.join(INPUT_DIR, ext.upper())))

if not image_files:
    print("❌ 没有找到图片，请先运行 Cell 3 上传图片")
else:
    print(f"📷 找到 {len(image_files)} 张图片")
    print(f"🎯 目标倍数: {TARGET_SCALE}x")
    print(f"📁 输出目录: {OUTPUT_DIR}")
    print("=" * 50)
    
    start_time = time.time()
    
    for idx, img_path in enumerate(image_files, 1):
        filename = os.path.basename(img_path)
        name, ext = os.path.splitext(filename)
        
        print(f"\n[{idx}/{len(image_files)}] 处理: {filename}")
        
        try:
            # 加载图片
            img = Image.open(img_path).convert('RGB')
            orig_size = img.size
            print(f"  📐 原始尺寸: {orig_size[0]}x{orig_size[1]}")
            
            # 放大
            upscaled = multi_scale_upscale(img, TARGET_SCALE)
            final_size = upscaled.size
            print(f"  ✅ 最终尺寸: {final_size[0]}x{final_size[1]}")
            
            # 保存 (保留原文件名 + 实际尺寸)
            output_filename = f"{name}_{final_size[0]}x{final_size[1]}{ext}"
            output_path = os.path.join(OUTPUT_DIR, output_filename)
            upscaled.save(output_path, quality=95)
            print(f"  💾 已保存: {output_filename}")
            
        except Exception as e:
            print(f"  ❌ 错误: {e}")
    
    elapsed = time.time() - start_time
    print("\n" + "=" * 50)
    print(f"🎉 全部完成! 耗时: {elapsed:.1f} 秒")
    print(f"📁 输出位置: {OUTPUT_DIR}")

In [ ]:
#@title 🔌 Cell 8: 断开运行时
from google.colab import runtime

print("⚠️ 即将断开运行时...")
print("📁 输出已保存到 Google Drive，无需等待下载")

runtime.unassign()